In [1]:
import torch

from torchmetrics.classification import MulticlassAccuracy

In [7]:
from utils_v2 import Flowers102Classifier, plot_training_runs, get_train_val_test_loader, FineTuneType, TrainingRun

In [8]:
train_loader, validation_loader, test_loader = get_train_val_test_loader()

TRAINING SIZE: 816
VALIDATION SIZE: 204
TRAINING SIZE: 6149


In [10]:
def transfer_learning_on_backbone(backbones, feature_extract_epochs, fine_tune_epochs):
    """Run transfer learning on multiple backbones for this classification task.
    The choice of the backbone (pre-trained model) is a hyper-parameter.
    
    We perform transfer-learning in 2 steps:
    1. Feature extraction, which is run for feature_extract_epochs, and
    2. Fine-tuning, which is run for fine_tune_epochs.
    
    We save the model with the best validation accuracy after every epoch.
    """
    device = "cuda" if torch.cuda.is_available() else "cpu"
    # Let's train the last classification later of the pre-trained model with the
    # specified backbone on the Flowers 102 dataset.
    
    training_runs = {}
    for backbone in backbones:
        best_cp_path = f'{backbone}_Flowers102_best.pt'
        print(f"Running feature extraction on a {backbone} backbone for {feature_extract_epochs} epochs.\n")
        fc = Flowers102Classifier(backbone=backbone)
        fc.to(device)

        optimizer = torch.optim.Adam(fc.parameters())
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=6, gamma=0.3)
        accuracy = MulticlassAccuracy(num_classes=102, average='micro').to(device)

        # First freeze all the weights except for the newly added Linear layer.
        fc.fine_tune(FineTuneType.NEW_LAYERS)

        best_test_accuracy = 0.0
        training_run = TrainingRun()
        training_runs[backbone] = training_run

        fc.train_multiple_epochs_and_save_best_checkpoint(
            train_loader,
            validation_loader,
            accuracy,
            optimizer,
            scheduler,
            feature_extract_epochs,
            best_cp_path,
            training_run,
        )
        
        print(f"Done with feature extraction for {backbone}-based model. Ran for {feature_extract_epochs} epochs.")
        
        best_val_accuracy = fc.get_metrics("val")['accuracy']
        print(f"[{backbone}] Best val accuracy after feature extraction is {best_val_accuracy}\n")
        print(f"Running fine-tuning for {fine_tune_epochs} epochs")

        # Set all the parameters to be trainable.
        fc.fine_tune(FineTuneType.ALL)

        optimizer = torch.optim.Adam(fc.get_optimizer_params(), lr=1e-8)
        # Every 2 steps reduce the LR to 70% of the previous value.
        scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.7)

        fc.train_multiple_epochs_and_save_best_checkpoint(
            train_loader,
            validation_loader,
            accuracy,
            optimizer,
            scheduler,
            fine_tune_epochs,
            best_cp_path,
            training_run,
        )
        print("-------------------------------------------------------------------------")
        print(f"Accuracy of {backbone}-based pre-trained model with last layer replaced.")
        fc.eval()
        fc.evaluate(test_loader, accuracy, 0, "Val")
        print("-------------------------------------------------------------------------")

    # end for (backbone)
    
    # Now plot the training runs.
    plot_training_runs(training_runs)
   
    
# end def
backbones = ["resnet18", "resnet50", "resnet152"]
transfer_learning_on_backbone(backbones, feature_extract_epochs=16, fine_tune_epochs=8)

Running feature extraction on a resnet18 backbone for 16 epochs.

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/queantran/opt/anaconda3/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3369, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/9w/jvgczv21237cq4w760m_n4vw0000gn/T/ipykernel_30100/3858712684.py", line 80, in <cell line: 80>
    transfer_learning_on_backbone(backbones, feature_extract_epochs=16, fine_tune_epochs=8)
  File "/var/folders/9w/jvgczv21237cq4w760m_n4vw0000gn/T/ipykernel_30100/3858712684.py", line 33, in transfer_learning_on_backbone
    fc.train_multiple_epochs_and_save_best_checkpoint(
  File "/Users/queantran/Desktop/Harry/SC4001/utils.py", line 202, in train_multiple_epochs_and_save_best_checkpoint
  File "/Users/queantran/Desktop/Harry/SC4001/utils.py", line 146, in train_one_epoch
    loss = criterion(outputs, targets)
  File "/Users/queantran/opt/anaconda3/lib/python3.9/site-packages/torch/nn/modules/module.py", line 1501, in _call_impl
    return forward_call(*arg